Biblioteki

In [25]:
import pandas as pd
from itertools import permutations

Słownik z punktami za dane miejsce w wyścigu

In [26]:
points_list = {1:25, 2:18, 3:15, 4:12, 5:10, 6:8,  7:6, 8:4, 9:2, 10:1, 11:0, 12:0}

Funkcja sprawdzająca kombinacje miejsc w wyścigu

In [27]:
def ChampionshipOutcomes(fastest_lap=None):
    
    unique_combinations = set()
    filtered_combinations = {}

    #Pętla generująca kombinacje
    for f1, f2, m1, m2 in permutations(range(1,13), 4):
        
        # Unikalne pary
        ferrari_pair = tuple(sorted([f1, f2]))
        mclaren_pair = tuple(sorted([m1, m2]))
        combination = (ferrari_pair, mclaren_pair)

        # Dodajemy tylko unikalne pary
        if combination not in unique_combinations:
            unique_combinations.add(combination)

            # Ignorujemy pary Ferrari [1, 12]
            if ferrari_pair == (1, 12):
                continue

            ferrari_points = points_list[f1] + points_list[f2]
            mclaren_points = points_list[m1] + points_list[m2]

            # Dodanie punktu za najszybsze okrążenie
            if fastest_lap == "Ferrari":
                ferrari_points += 1
            elif fastest_lap == "McLaren":
                # McLaren ląduje na niepunktowanych pozycjach
                if not (m1 == 11 and m2 == 12):
                    mclaren_points += 1

            # Sprawdzenie czy Ferrari wygrało wyścig
            ferrari_has_first = 1 in (f1, f2)
            required_difference = 21 if ferrari_has_first else 22

            # Wybranie tych które dają mistrzostwo Ferrari
            if (ferrari_points - mclaren_points) >= required_difference:
                if ferrari_pair not in filtered_combinations:

                    filtered_combinations[ferrari_pair] = {
                    "Places":[f1, f2, m1, m2],
                    "Ferrari points": ferrari_points,
                    "McLaren points": mclaren_points,
                    "Difference": ferrari_points - mclaren_points,
                }
                else:
                    # Wybranie tych z maksymalną pozycją McLarena
                    current_mclaren_places = filtered_combinations[ferrari_pair]["Places"][2:]
                    if max(mclaren_pair) < max(current_mclaren_places):
                        filtered_combinations[ferrari_pair] = {
                            "Places":[f1, f2, m1, m2],
                            "Ferrari points": ferrari_points,
                            "McLaren points": mclaren_points,
                            "Difference": ferrari_points - mclaren_points
                }
    # Lista z kombinacjami miejsc                                   
    return list(filtered_combinations.values())

Funkcja tabeli

In [28]:
def generate_table(outcomes):
    # DataFrame
    df = pd.DataFrame(outcomes)

    # Rozbice tabeli "Places"
    df['Ferrari'] = df['Places'].apply(lambda x: f"{x[0]} & {x[1]}")
    df['McLaren'] = df['Places'].apply(lambda x: f"{x[2]} & {x[3]}")
    df = df.drop(columns='Places')

    # Sortowanie według punktow McLarena
    df = df.sort_values(by='McLaren points', ascending=False)

    # Ustalanie kolejności kolumn
    column_order = ['Ferrari', 'McLaren', 'Difference', 'Ferrari points', 'McLaren points']
    df = df[column_order]

    return df

Wyświetlenie danych możliwości

In [29]:
#Żadna ekpia nie ma najszybszego okrążenia
no_fastest_lap = ChampionshipOutcomes()
nobody_outcomes_table = generate_table(no_fastest_lap)
display(nobody_outcomes_table.style.hide(axis='index'))

Ferrari,McLaren,Difference,Ferrari points,McLaren points
1 & 2,4 & 5,21,43,22
1 & 3,5 & 6,22,40,18
1 & 4,5 & 7,21,37,16
1 & 5,6 & 7,21,35,14
1 & 7,6 & 9,21,31,10
2 & 3,7 & 8,23,33,10
1 & 6,7 & 8,23,33,10
1 & 8,7 & 9,21,29,8
2 & 4,7 & 9,22,30,8
2 & 5,8 & 9,22,28,6


In [30]:
# Ferrari zdobywa najszybsze okrążenie
ferrari_fastest_lap = ChampionshipOutcomes(fastest_lap="Ferrari")
ferrari_outcomes_table = generate_table(ferrari_fastest_lap)
display(ferrari_outcomes_table.style.hide(axis='index'))

Ferrari,McLaren,Difference,Ferrari points,McLaren points
1 & 2,4 & 5,22,44,22
1 & 3,4 & 6,21,41,20
1 & 4,5 & 7,22,38,16
1 & 5,6 & 7,22,36,14
2 & 3,6 & 8,22,34,12
1 & 7,6 & 9,22,32,10
1 & 6,7 & 8,24,34,10
1 & 8,7 & 9,22,30,8
2 & 4,7 & 9,23,31,8
1 & 9,7 & 10,21,28,7


In [31]:
# McLaren zdobywa najszybsze okrążenie
mclaren_fastest_lap = ChampionshipOutcomes(fastest_lap="McLaren")
mclaren_outcomes_table = generate_table(mclaren_fastest_lap)
display(mclaren_outcomes_table.style.hide(axis='index'))

Ferrari,McLaren,Difference,Ferrari points,McLaren points
1 & 2,4 & 6,22,43,21
1 & 3,5 & 6,21,40,19
1 & 4,6 & 7,22,37,15
1 & 5,6 & 8,22,35,13
2 & 3,7 & 8,22,33,11
1 & 6,7 & 8,22,33,11
1 & 8,7 & 10,21,29,8
1 & 7,8 & 9,24,31,7
2 & 4,8 & 9,23,30,7
1 & 9,8 & 10,21,27,6


Export danych do plików CSV

In [ ]:
ferrari_outcomes_table.to_csv("Fastest lap Ferrari.csv", index=False)
nobody_outcomes_table.to_csv("Without fastest lap.csv", index=False)
mclaren_outcomes_table.to_csv("Fastest lap McLaren.csv", index=False)